In [219]:
from pypdf import PdfReader
name = 'Jongkook Kim'

summary = ''
with open('me/summary.txt', 'r', encoding='utf-8') as file:
    summary = file.read()

linkedin = ''
linkedin_profile = PdfReader('me/Profile.pdf')
for page in linkedin_profile.pages:
    text = page.extract_text()
    if text:
        linkedin += text


In [220]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from openai import OpenAI


In [221]:

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    avator_response: str


In [222]:
avator_system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

avator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
avator_system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

def avator(user_question, history, evaluation: Evaluation): 
    system_prompt = ''
    if 'xyz' in user_question:
        print("force to fail")
        system_prompt = "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    
    if evaluation != None and not evaluation.is_acceptable:
        print(f"{evaluation.avator_response} is not acceptable. Retry")
        system_prompt = avator_system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
        system_prompt += f"## Your attempted answer:\n{evaluation.avator_response}\n\n"
        system_prompt += f"## Reason for rejection:\n{evaluation.feedback}\n\n"

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role":"user", "content": user_question}]

    llm_client = OpenAI().chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    
    return llm_client.choices[0].message.content

In [223]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

def evaluator(user_question, avator_response, history) -> Evaluation:
    messages = [{'role':'system', 'content': evaluator_system_prompt}] + [{'role':'user', 'content':evaluator_user_prompt(reply=avator_response, message=user_question, history=history)}]

    llm_client = OpenAI(api_key=os.getenv('GOOGLE_API_KEY'), base_url='https://generativelanguage.googleapis.com/v1beta/openai/')
    response = llm_client.beta.chat.completions.parse(model='gemini-2.0-flash',messages=messages,response_format=Evaluation)

    evaluation = response.choices[0].message.parsed

    evaluation.avator_response = avator_response
    return evaluation

In [224]:
max_evaluate = 2
def orchestrator(message, history):
    avator_response = avator(message, history, None)
    for occurrence in range(1, max_evaluate+1):
        if occurrence <= max_evaluate:
            evaluation = evaluator(user_question=message, avator_response=avator_response, history=history)
            if not evaluation.is_acceptable:
                message_with_feedback = evaluation.feedback + message
                avator_response = avator(message_with_feedback, history, evaluation)

    return avator_response


In [ ]:
import gradio
gradio.ChatInterface(orchestrator, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


I don’t work for any specific organization, including Morgan Stanley. I'm here to provide information and answer questions to the best of my abilities. How can I assist you today? is not acceptable. Retry
force to fail
force to fail
owhay aboutyay xyz? is not acceptable. Retry
force to fail
I apologize for the confusion, and thank you for your understanding. To clarify, I currently hold a position at Morgan Stanley as a Lead Software Engineer. In this capacity, I am responsible for designing and architecting distributed data service systems while ensuring they operate with high availability and low latency. If you have any questions about my specific experiences or projects at Morgan Stanley, I would be happy to provide more information. is not acceptable. Retry
force to fail
force to fail
ay-nay, I-way ave-hay ot-nay orked-way orf-way at-way xyz-way ompany-cay. is not acceptable. Retry
